In [ ]:
%matplotlib inline
# -*- coding: utf-8 -*-
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from odin.common import utils

project_name = 'biz_req'

dp = utils.get_dataframe_by_project_name(project_name)
redmine = utils.redmine_connect()

In [ ]:
"""
Initialize basic DataFrame and set
"""

dp_bug = dp[dp.tracker == u'错误']
dp_bug.loc[:,'interval'] = dp_bug.updated_on - dp_bug.created_on
dp_bug_resovled = dp_bug[dp_bug.status == u'已解决']
custom_field_feature_id = utils.get_custom_field_id_by_name(u'功能模块')
custom_field_version_id = utils.get_custom_field_id_by_name(u'版本信息')

In [ ]:
"""
X: datetime
Y: # of issue and show the status composition
"""

d = {}
for i in redmine.issue_status.all():
    dp_tmp = dp_bug[dp_bug.status == i.name].groupby('created_on').size()
    d[i.name] = dp_tmp

total = pd.DataFrame(d)
total['2017-8-18':'2017-8-26'].plot.bar(stacked=True)

In [ ]:
"""
X: feature
Y: # of issue and show the status composition
"""

d = {}
for i in redmine.custom_field.all().get(custom_field_feature_id)['possible_values']:
    i['value'] = i['value'].strip('-')
    dp_tmp = dp_bug[dp_bug.module == i['value']].groupby('status').size()
    d[i['value']] = dp_tmp
total = pd.DataFrame(d)
total.T.plot.bar(stacked=True, figsize=(20, 6))

In [ ]:
"""
X: version
Y: # of issue and show the status composition
"""

d = {}
for i in redmine.custom_field.all().get(custom_field_version_id)['possible_values']:
    dp_tmp = dp_bug[dp_bug.version == i['value']].groupby('status').size()
    d[i['value']] = dp_tmp
total = pd.DataFrame(d)
total.T.plot.bar(stacked=True)

In [ ]:
"""
需求耗费时长:
(1)单一需求耗费时长=需求上线时间-需求提出时间
(2)需求耗费时长=百分位数（所有单一需求耗费时长，85%）
(3)需求耗费时长分布K值：需求耗费时长分布K值反应需求耗费时长的分布特征。为了计算需求耗费时长分布
K值，需要先绘制需求耗费时长分布图。分布图是一个柱状图，横轴上X位置柱状高度是需求耗费时长为X天
的需求的个数。符合韦伯分布（Weibull Distribution）
"""

dp_bug_resovled.groupby('interval').size().plot()


In [ ]:
"""
X: resovled duration
Y: # of issue and show the feature composition
"""

d = {}
for i in redmine.custom_field.all().get(custom_field_feature_id)['possible_values']:
    i['value'] = i['value'].strip('-')
    dp_tmp = dp_bug_resovled[dp_bug_resovled.module == i['value']].groupby('interval').size()
    d[i['value']] = dp_tmp
total = pd.DataFrame(d)
total.plot.bar(stacked=True)

In [ ]:
"""
X: resovled duration
Y: # of issue and show the version composition
"""

d = {}
for i in redmine.custom_field.all().get(custom_field_version_id)['possible_values']:
    dp_tmp = dp_bug_resovled[dp_bug_resovled.version == i['value']].groupby('interval').size()
    d[i['value']] = dp_tmp
total = pd.DataFrame(d)
total.plot.bar(stacked=True)